In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

### Загрузим датасет с машинами. Цель - верно восстанавливать для каждой из них цену продажи!

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/18/autos.csv')

data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [4]:
### Колонка с тергетом - "selling price"

X = data.drop("selling_price", axis=1)
y = data["selling_price"]

### Будем замерять MSLE!
### Поэтому прологарифмируем таргет
### А после оптимизируем MSE

y = y.apply(np.log1p)

In [5]:
### Разделим выборку на трейн и тест!

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

__Задание__

Реализуйте свой MeanTargetEncoder с добавленем некоторого шума!

Однажды в лекционном материале, обсуждая счетчики, мы говорили с вами о том, что из-за них модели могут переобучаться. Один из способов бороться с этим - валидировать расчеты среднего таргета (стратегия отложенной выборки / расчеты на кросс-валидации). Но есть еще проще!

Можно просто к значению счетчика добавить случайный шум (зашумить данные)!

Напомним, что рассчитываться новые признаки должны по такой формуле:

$$
g_j = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}{l} + C * \epsilon
$$



Пусть шум будет случайной величиной из нормального стандартного распределения, то есть $\epsilon \sim N(0, 1) $, а $ C = 0.006$.

Создавая свой класс-трансформер, наследуйтесь от классов `BaseEstimator, TransformerMixin` из `sklearn.base`. Трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики нужно считать только по обучающей выборке в методе `fit`. Ваш трансформер должен принимать при инициализации список из категориальных признаков и список из числовых признаков.

Если для какого-то признака в тестовой выборке отсутствует значение, трансформер должен поставить там 0.

На выходе должен получиться датасет того же размера с измененными категориальными признаками.

Класс MeanTargetEncoderNoise должен иметь следующую сигнатуру:



In [ ]:
X.describe(include='object')

,name,fuel,seller_type,transmission,owner
count,4340,4340,4340,4340,4340
unique,1491,5,3,2,5
top,Maruti Swift Dzire VDI,Diesel,Individual,Manual,First Owner
freq,69,2153,3244,3892,2832


In [ ]:
X.describe(include='object').columns

Index(['name', 'fuel', 'seller_type', 'transmission', 'owner'], dtype='object')

In [ ]:
from random import random

mean_values = dict()
data_2 = data.copy()
for col in data_2.describe(include='object').columns:
  mean_map = data_2.groupby(col).agg({'selling_price' :'mean'}).map(lambda x: x + random() * 0.006).to_dict()['selling_price']
  data_2[col] = data_2[col].map(mean_map)
data_2.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,94347.830610,2007,60000,70000,344840.140036,424505.421111,400066.691027,598636.974377
1,144416.672579,2007,135000,50000,344840.140036,424505.421111,400066.691027,598636.974377
2,490666.401800,2012,600000,100000,669094.252780,424505.421111,400066.691027,598636.974377
3,229999.860367,2017,250000,46000,344840.140036,424505.421111,400066.691027,598636.974377
4,530555.559229,2014,450000,141000,669094.252780,424505.421111,400066.691027,343891.094446


In [ ]:
from random import random

mean_values = dict()
data_2 = data.copy()
for col in data_2.describe(include='object').columns:
  mean_map = data_2.groupby(col).agg({'selling_price' :'mean'}).map(lambda x: x + random() * 0.006).to_dict()['selling_price']
  data_2[col] = data_2[col].map(mean_map)
data_2.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,94347.826087,2007,60000,70000,344840.137541,424505.419236,400066.685766,598636.969633
1,144416.666667,2007,135000,50000,344840.137541,424505.419236,400066.685766,598636.969633
2,490666.400000,2012,600000,100000,669094.252206,424505.419236,400066.685766,598636.969633
3,229999.857143,2017,250000,46000,344840.137541,424505.419236,400066.685766,598636.969633
4,530555.555556,2014,450000,141000,669094.252206,424505.419236,400066.685766,343891.088608


In [ ]:
data_2.groupby(col).agg({y.name :'mean'})

,selling_price
owner,
173901.202302,173901.197531
269474.004554,269474.003289
343891.094446,343891.088608
598636.974377,598636.969633
954293.944310,954293.941176


In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
from random import random

class MeanTargetEncoderNoiseS(BaseEstimator, TransformerMixin):

    def __init__(self, categorical, numeric):
        self.cat = categorical
        self.num = numeric

        self.noise_const = 0.006

    def fit(self, X, y):
        self.target = y.name

        X_ = X.copy()
        y_ = y.copy()

        X_with_target = pd.concat((X_, y_), axis=1)

        # словарик - значение колонки: среднее значение таргета + шум
        self.mean_map = {}
        for col in X_with_target[self.cat]:

          self.mean_map = X_with_target.groupby(col).agg({y.name :'mean'})\
          .map(lambda x: x + random() * self.noise_const)\
          .to_dict()[y.name]

          X_[col] = X_[col].map(self.mean_map)

        self.X_fit = X_

        return self

    def transform(self, df):
        # df_ = df_.drop(columns=self.target)



        return self.X_fit

Разделите колонки на вещественные и категориальные. Приведите все категориальные колонки к типу `object`.

Далее применим наш кодировщик к `X_train, X_test`, так же как например мы применяем `StandardScaler`, чтобы проверить работоспособность нашего класса. Установите зерно датчика случайный чисел `np.random.seed(1)`.

После того, как вы изменили обучающую и тестовую выборки, сохраните первые 10 строк полученного промежуточного датафрейма обучающей выборки (`X_train`) в файл в формате csv с сепаратором `;`. Не забудьте индекс. Отправьте полученный файл в форму ниже.

Список колонок которые должны быть в файле для сдачи:
```py
cols = [
    "km_driven",
    "name",
    "year",
    "fuel",
    "seller_type",
    "transmission",
    "owner"
]
```

### Ваше решение


Разделение колонок на категориальные и числовые.

In [6]:
object_cols = ['name', 'year', 'fuel', 'seller_type', 'transmission', 'owner']
num_cols = ['km_driven']

X.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,141000,Diesel,Individual,Manual,Second Owner


Реализация класса MeanTargetEncoderNoise.

In [8]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
    def __init__(self, categorical, numeric, C=0.006):
        self.categorical = categorical
        self.numeric = numeric
        self.C = C
        self.mean_target = {}

    def fit(self, X, y):
        df = X.copy()
        df['selling_price'] = y
        self.global_mean = y.mean()
        for col in self.categorical:
            means = df.groupby(col)['selling_price'].mean()
            noise = np.random.normal(0, 1, size=means.shape[0])
            self.mean_target[col] = (means + self.C * noise).to_dict()
        return self

    def transform(self, df):
        temp = df.copy()
        for col in self.categorical:
            temp[col] = temp[col].map(self.mean_target.get(col, {})).fillna(0)
        return temp[self.categorical + self.numeric]


Проверка работы трансформера.

In [11]:
np.random.seed(1)
transformer = MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)
transformer_s = MeanTargetEncoderNoiseS(categorical=object_cols, numeric=num_cols)

transformer.fit(X_train, y_train)
transformer_s.fit(X_train, y_train)

train = transformer.transform(X_train)
test = transformer.transform(X_test)

train_s = transformer_s.transform(X_train)

train.head(10)

,name,year,fuel,seller_type,transmission,owner,km_driven
3294,13.483692,13.436559,13.093756,12.615863,13.771135,12.977730,50000
2290,12.117029,11.903115,12.453832,12.615863,13.771135,12.977730,70000
874,12.310456,13.328864,12.453832,12.615863,12.639805,12.977730,50000
1907,12.491443,13.042359,12.453832,13.152824,12.639805,12.463313,92198
3244,12.390906,12.870886,12.453832,12.615863,12.639805,12.463313,3240
1089,12.687432,13.436559,12.453832,13.152824,12.639805,12.977730,10000
3902,11.698702,11.503504,12.453832,12.615863,12.639805,11.878390,90000
2215,11.120678,11.503504,12.453832,12.615863,12.639805,12.463313,79000
3862,13.172879,13.328864,13.093756,12.615863,12.639805,12.977730,99700
705,13.004194,12.241213,13.093756,12.615863,12.639805,12.463313,124000


In [12]:
train_s.head(10)

,name,year,km_driven,fuel,seller_type,transmission,owner
3294,13.473367,13.441349,50000,13.089180,12.620320,13.771376,12.978173
2290,12.116263,11.914640,70000,12.458481,12.620320,13.771376,12.978173
874,12.302591,13.341379,50000,12.458481,12.620320,12.642099,12.978173
1907,12.489154,13.057857,92198,12.458481,13.149074,12.642099,12.465627
3244,12.396526,12.863167,3240,12.458481,12.620320,12.642099,12.465627
1089,12.691550,13.441349,10000,12.458481,13.149074,12.642099,12.978173
3902,11.697745,11.505381,90000,12.458481,12.620320,12.642099,11.874915
2215,11.121988,11.505381,79000,12.458481,12.620320,12.642099,12.465627
3862,13.172919,13.341379,99700,13.089180,12.620320,12.642099,12.978173
705,13.000675,12.249888,124000,13.089180,12.620320,12.642099,12.465627


In [ ]:
train.head(10).reset_index(drop=True).to_csv("HW_18_2.csv", sep=';', index=False)

In [ ]:
train.columns

Index(['name', 'year', 'km_driven', 'fuel', 'seller_type', 'transmission',
       'owner'],
      dtype='object')

Обучите несколько деревьев, перебирая максимальную глубину алгоритма из списка `max_depth_list`, а остальные параметры оставьте дефолтными. Выведите лучшее значение гиперпараметра. Постройте график зависимости MSLE на тестовой выборке от значения гиперпараметра. Воспользуйтесь `Pipeline` без `GridSearch`. Проделайте то же самое с `min_samples_split`, `min_impurity_decrease`, `max_leaf_nodes`. (по 2б на каждый параметр)

In [13]:
max_depth_list = [3, 5, 8, 12]
min_samples_split_list = [10, 50, 100, 500]
min_impurity_decrease_list = [0, 0.1, 0.15, 0.2]
max_leaf_nodes_list = [100, 200, 500]

In [14]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

np.random.seed(1)

### Your code is here

mse_dict = {}
mse_list = []
for depth in max_depth_list:
  pipe = Pipeline([
      ('scaler', MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
      ('model', DecisionTreeRegressor(max_depth=depth))
      ])
  pipe.fit(X_train, y_train)
  score = pipe.score(X_test, y_test)

  mse_dict[depth] = score
  mse_list.append(score)



In [15]:
mse_dict

{3: -0.1383237202056966,
 5: -1.0555948659230725,
 8: -1.8358455741088506,
 12: -1.8316609277794167}

In [16]:
mse_dict = {}
for samples_split in min_samples_split_list:
  pipe = Pipeline([
      ('scaler', MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
      ('model', DecisionTreeRegressor(min_samples_split=samples_split))
      ])
  pipe.fit(X_train, y_train)

  score = pipe.score(X_test, y_test)

  mse_dict[depth] = score
  mse_list.append(score)

print(mse_dict)

{12: -0.1527880686367593}


In [17]:
mse_dict = {}
for impurity in min_impurity_decrease_list:
  pipe = Pipeline([
      ('scaler', MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
      ('model', DecisionTreeRegressor(min_impurity_decrease=impurity))
      ])
  pipe.fit(X_train, y_train)

  score = pipe.score(X_test, y_test)

  mse_dict[depth] = score
  mse_list.append(score)

print(mse_dict)

{12: 0.25856596474496174}


In [18]:
mse_dict = {}
for leaf_nodes in max_leaf_nodes_list:
  pipe = Pipeline([
      ('scaler', MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
      ('model', DecisionTreeRegressor(max_leaf_nodes=leaf_nodes))
      ])
  pipe.fit(X_train, y_train)

  score = pipe.score(X_test, y_test)

  mse_dict[depth] = score
  mse_list.append(score)

print(mse_dict)

{12: -1.8334360345104015}


In [23]:
list(map(lambda x: abs(x),mse_list))

[0.1383237202056966,
 1.0555948659230725,
 1.8358455741088506,
 1.8316609277794167,
 1.0454491291514416,
 1.0519667681022327,
 0.35860868420884384,
 0.1527880686367593,
 1.8361970945883987,
 0.25906392809688183,
 0.2573793760822848,
 0.25856596474496174,
 1.8365307242817366,
 1.8328873610440275,
 1.8334360345104015]

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error


np.random.seed(1)

def evaluate_model(pipe, X_train, y_train, X_test, y_test):
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    msle = mean_squared_error(y_test, preds)
    return msle

max_depth_list = [3, 5, 8, 12]
msle_results_depth = []

for max_depth in max_depth_list:
    pipe = Pipeline([
        ("custom_transformer", MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
        ("decision_tree", DecisionTreeRegressor(max_depth=max_depth))
    ])

msle = evaluate_model(pipe, X_train, y_train, X_test, y_test)
msle_results_depth.append(msle)

# Выводим результаты
for depth, msle in zip(max_depth_list, msle_results_depth):
    print(f'Max Depth: {depth}, MSLE: {msle:.2f}')

# Находим наименьшее значение MSLE
best_max_depth = max_depth_list[np.argmin(msle_results_depth)]
best_msle_depth = min(msle_results_depth)
print(f'Лучший max_depth: {best_max_depth} с MSLE: {best_msle_depth:.2f}')

min_samples_split_list = [10, 50, 100, 500]
msle_results_split = []

for min_samples_split in min_samples_split_list:
    pipe = Pipeline([
        ("custom_transformer", MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
        ("decision_tree", DecisionTreeRegressor(min_samples_split=min_samples_split))
    ])

msle = evaluate_model(pipe, X_train, y_train, X_test, y_test)
msle_results_split.append(msle)

# Выводим результаты
for split, msle in zip(min_samples_split_list, msle_results_split):
    print(f'Min Samples Split: {split}, MSLE: {msle:.2f}')

best_min_samples_split = min_samples_split_list[np.argmin(msle_results_split)]
best_msle_split = min(msle_results_split)
print(f'Лучший min_samples_split: {best_min_samples_split} с MSLE: {best_msle_split:.2f}')

min_impurity_decrease_list = [0, 0.1, 0.15, 0.2]
msle_results_impurity = []

for min_impurity_decrease in min_impurity_decrease_list:
    pipe = Pipeline([
        ("custom_transformer", MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
        ("decision_tree", DecisionTreeRegressor(min_impurity_decrease=min_impurity_decrease))
    ])

msle = evaluate_model(pipe, X_train, y_train, X_test, y_test)
msle_results_impurity.append(msle)

# Выводим результаты
for impurity, msle in zip(min_impurity_decrease_list, msle_results_impurity):
    print(f'Min Impurity Decrease: {impurity}, MSLE: {msle:.2f}')

best_min_impurity_decrease = min_impurity_decrease_list[np.argmin(msle_results_impurity)]
best_msle_impurity = min(msle_results_impurity)
print(f'Лучший min_impurity_decrease: {best_min_impurity_decrease} с MSLE: {best_msle_impurity:.2f}')

max_leaf_nodes_list = [100, 200, 500]
msle_results_leaf = []

for max_leaf_nodes in max_leaf_nodes_list:
    pipe = Pipeline([
        ("custom_transformer", MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
        ("decision_tree", DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes))
    ])

msle = evaluate_model(pipe, X_train, y_train, X_test, y_test)
msle_results_leaf.append(msle)

# Выводим результаты
for leaf, msle in zip(max_leaf_nodes_list, msle_results_leaf):
    print(f'Max Leaf Nodes: {leaf}, MSLE: {msle:.2f}')

best_max_leaf_nodes = max_leaf_nodes_list[np.argmin(msle_results_leaf)]
best_msle_leaf = min(msle_results_leaf)
print(f'Лучший max_leaf_nodes: {best_max_leaf_nodes} с MSLE: {best_msle_leaf:.2f}')


Max Depth: 3, MSLE: 1.98
Лучший max_depth: 3 с MSLE: 1.98
Min Samples Split: 10, MSLE: 0.81
Лучший min_samples_split: 10 с MSLE: 0.81
Min Impurity Decrease: 0, MSLE: 0.52
Лучший min_impurity_decrease: 0 с MSLE: 0.52
Max Leaf Nodes: 100, MSLE: 1.98
Лучший max_leaf_nodes: 100 с MSLE: 1.98


Подберите лучшую комбинацию параметров, используя `GridSearchCV` и набор массивов значений параметров из предыдущего задания. Для лучшей комбинации посчитайте MSLE на тестовой выборке. Получились ли лучшие параметры такими же, как если бы вы подбирали их по-отдельности при остальных гиперпараметрах по умолчанию (предыдущее задание)? (2б)

In [25]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "decision_tree__max_depth": [3, 5, 8, 12],
    "decision_tree__min_samples_split": [10, 50, 100, 500],
    "decision_tree__min_impurity_decrease": [0, 0.1, 0.15, 0.2],
    "decision_tree__max_leaf_nodes": [100, 200, 500]
}
np.random.seed(1)

### Your code is here
pipe = Pipeline([
    ('encoder', MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
    ('decision_tree', DecisionTreeRegressor())
])

grid = GridSearchCV(pipe, param_grid)

grid.fit(X_train, y_train)




GridSearchCV(estimator=Pipeline(steps=[('encoder',
                                        MeanTargetEncoderNoise(categorical=['name',
                                                                            'year',
                                                                            'fuel',
                                                                            'seller_type',
                                                                            'transmission',
                                                                            'owner'],
                                                               numeric=['km_driven'])),
                                       ('decision_tree',
                                        DecisionTreeRegressor())]),
             param_grid={'decision_tree__max_depth': [3, 5, 8, 12],
                         'decision_tree__max_leaf_nodes': [100, 200, 500],
                         'decision_tree__min_impurity_decrease': [0, 0.1, 0.15,
                                                                  0.2],
                         'decision_tree__min_samples_split': [10, 50, 100,
                                                              500]})

In [26]:
grid.best_params_

{'decision_tree__max_depth': 12,
 'decision_tree__max_leaf_nodes': 200,
 'decision_tree__min_impurity_decrease': 0.1,
 'decision_tree__min_samples_split': 100}

In [27]:
grid.best_score_

np.float64(0.18985951154026395)

In [28]:
mean_squared_error(y_test, grid.best_estimator_.predict(X_test))

0.520009858723521